## Optimizing case definitions
Public health case definitions often take the form of predictive checklists, like "if the person has 2 or more of the following 5 symptoms, then he or she is a probable case". Case definitions can be built by hand, e.g., by using published data and expert intuition to design them from scratch, but they can also be built automatically. This notebook provides a brief overview of the latter, as enabled by the `cdo` Python package.

### Checking the performance of the original predictors
Before trying to find optimal combinations of symptoms, it's often helpful to look at the performance of each symptom individually. Let's load the test data we have in the repository and see what predictors we have to work with.

In [ ]:
import numpy as np
import pandas as pd
import time
import seaborn as sns

from matplotlib import pyplot as plt
from importlib import reload

from cdo import optimizers as ops
from cdo import metrics, tools

In [ ]:
records = pd.read_csv('data/stroke_data.csv')
records.head()

Here, we have some synthetic data about a particular outcome (`stroke`) and some of its predictors (the rest of the columns). Let's see how strongly each of the individual predictors is associated with the outcome. 

In [ ]:
y = records.stroke.values
X = records.iloc[:, records.columns != 'stroke']

single_stats = pd.concat([metrics.clf_metrics(y, X[c], mod_name=c) for c in X.columns.values])
single_stats.columns.values[0] = 'symptom'
single_stats.sort_values('j', ascending=False)

That's a lot of numbers! Focusing on Youden's J index (`j`), though, which is a good overall measure of classification performance, shows that there are a number of strong predictors, including hyptertension (`high_bp`), high red blood cell count (`high_rbc`), and sedentary lifestyle (`sedentary`). Picking hypertension alone would have decent performance as a screening rule (sensitivity and specificity are 84% and 73%, respectively), but let's see if we can do better by looking at combinations of the predictors.

## Examining simple combinations
The `cdo` package implements three main methods for evaluating simple combinations of symptoms (or other kinds of predictors): an integer program, a nonlinear approximation to an integer program, and a brute-force combinatorial search. Since the integer program is guaranteed to find the best solution, let's start there.

In [ ]:
ip = ops.IntegerProgram()
ip.fit(X, y)

Notice how easy it is to fit the optimizer--all you need to do is pass the predictors as a `pd.DataFrame` to the .fit() method, along with the outcome as an `np.array`, and it will find the (again, guaranteed optimal) solution for you. Let's see what it was by checking the `optimizer.results` object.

In [ ]:
ip.results

We can see that Youden's J index has moved up to 0.71 from 0.57--not bad! Relative to `high_bp`, we gained a little specificity, and we gained a ton of sensitivity, such that the new rule only has a 4% false-negative rate. What *is* the new rule, though? Since this results format is used for the other optimizers, let's take a second to see how we can use the column names of the results object to reconstruct a plain-language case definition.

In [ ]:
best_simple = tools.rule_df_to_str(ip.results)
best_simple

Cool. So we can see that, in the `ip.results` DataFrame, `m1` is the minimum number of symptoms the person must have, and `rule1` is the collection of symptoms. The third column, `n1`, specifies the total number of symptoms under consideration in this particular combination. Each of these has a `1` at the end because, as we'll see later, some of the optimizers also let you search for compound combinations, or combinations of simple rules, and so there will be two sets of `m` values, `n` values, and `rule` strings.

The other two optimizers, `NonlinearApproximation` and `FullEnumeration`, will also work for simple combinations. Let's start with the nonlinear approximation and see how it does.

In [ ]:
nola = ops.NonlinearApproximation()
nola.fit(X, y)
nola.results

So the runtime here was super fast--less than a second!--but we can see that with a J of 0.64, the solution is not as good as the optimum found by the integer program (both are mathematically set to optimize J). This is the downside to the nonlinear approximation--the math it uses to approximate the integer program runs fast, but it's not guaranteed to be optimal (or even close).

Finally, let's try the brute-force approach. As long as we don't limit the maximum number of symptoms `n` in any candidate combination, it should recover the solution from the integer program, along with many others.

In [ ]:
fe = ops.FullEnumeration()
fe.fit(X, y, max_n=6, verbose=False)

In [ ]:
fe.results

There we go! Crunching through all of the potential combinations recovered the optimal solution from before, and it also got us all of the other ones (all 20,617) of them. Having the full enumeration is reassuring in that it confirms the solution from the integer program, but it also lets us do two important things:

1. Visualize the distribution of metrics for all the combinations; and
2. Pick our "top" combination by examining criteria other than just the metric that was optimized.

Visualization is straightforward and is handled by the `FullEnumeration.plot()` function. Since we optimized `j`, let's look at the candidate combinations as points in ROC space.

In [ ]:
fe.plot()

Not very exciting, but nice if you want to get a sense for how much variability there was in the combinations' performance. We can also do a panel plot that sorts the combinations by the total number of symptoms they considered (`total_n` in the `results` object).

In [ ]:
fe.plot(separate_n=True)

Again, these aren't very exciting, partly because the symptoms in our test data are synthetic and have less variability than they might if they were real, but they're still handy.

